<a href="https://colab.research.google.com/github/aritrasen87/smolagents/blob/main/2_hf_smolagents_agentic_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Installations

In [1]:
! pip -q install smolagents
! pip -q install litellm
! pip install langchain langchain-community rank_bm25 --upgrade -q
! pip install sentence-transformers -q
! pip install datasets -q
! pip install chromadb -q


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.1/68.1 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.4/321.4 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 111.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.7/81.7 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 89.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 106.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the sour

### Credentials

In [2]:
import os
from google.colab import userdata
os.environ['HF_TOKEN'] = userdata.get('HF_TOKEN')
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_KEY')

### Testing smolagents

In [3]:
from smolagents import CodeAgent, HfApiModel

model = HfApiModel()

agent = CodeAgent(tools=[],model=model)

agent.run('What is 24*365?')

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_config.py:345: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed
  warnings.warn(message, UserWarning)


╭────────────────────────────────────────────────────────────────────────────────────────────── New run ───────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                                                                                                      │
│ What is 24*365?                                                                                                                                                                                      │
│                                                                                                                                                                                                      │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 0 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing this code: ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── 
  result = 24 * 365                                                                                                                                                                                     
  final_answer(result)                                                                                                                                                                                  
 ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: 8760

[Step 0: Duration 2.25 seconds| Input tokens: 1,985 | Output tokens: 68]

8760

### Vanilla RAG has limitations, most importantly these two:

1. It performs only one retrieval step: if the results are bad, the generation in turn will be bad.

2.  The user query will often be a question and the document containing the true answer will be in affirmative voice, so its similarity score will be downgraded compared to other source documents in the interrogative form, leading to a risk of missing the relevant information.

This Agent will:
- ✅ Formulate the query itself
- ✅ Critique to re-retrieve if needed.

### Indexing data into Chroma

In [4]:
import datasets
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain.docstore.document import Document
#from langchain_community.retrievers import BM25Retriever

knowledge_base = datasets.load_dataset("m-ric/huggingface_doc", split="train")
knowledge_base = knowledge_base.filter(lambda row: row["source"].startswith("huggingface/transformers"))

source_docs = [
    Document(page_content=doc["text"], metadata={"source": doc["source"].split("/")[1]})
    for doc in knowledge_base
]

### Creating Chunks using RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    add_start_index=True,
    strip_whitespace=True,
    separators=["\n\n", "\n", ".", " ", ""],
)
new_docs = text_splitter.split_documents(documents=source_docs)

###  BGE Embddings

from langchain_community.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-small-en"
model_kwargs = {"device": "cuda"}
encode_kwargs = {"normalize_embeddings": True}
embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)
### Populate Vector DB

db = Chroma.from_documents(new_docs, embeddings)

README.md:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

huggingface_doc.csv:   0%|          | 0.00/22.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2647 [00:00<?, ? examples/s]

Filter:   0%|          | 0/2647 [00:00<?, ? examples/s]

<ipython-input-4-b60c3d7902aa>:33: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceBgeEmbeddings(


...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [11]:
retriever = db.as_retriever(search_kwargs={"k": 4})
retriever.invoke('forward pass in transformer')

[Document(metadata={'source': 'transformers', 'start_index': 7510}, page_content='4.  [ ] Created script that successfully runs forward pass using\n    original repository and checkpoint\n\n5.  [ ] Successfully opened a PR and added the model skeleton to Transformers\n\n6.  [ ] Successfully converted original checkpoint to Transformers\n    checkpoint\n\n7.  [ ] Successfully ran forward pass in Transformers that gives\n    identical output to original checkpoint\n\n8.  [ ] Finished model tests in Transformers\n\n9.  [ ] Successfully added Tokenizer in Transformers'),
 Document(metadata={'source': 'transformers', 'start_index': 7863}, page_content='4.  [ ] Created script that successfully runs forward pass using\n    original repository and checkpoint\n\n5.  [ ] Successfully opened a PR and added the model skeleton to Transformers\n\n6.  [ ] Successfully converted original checkpoint to Transformers\n    checkpoint\n\n7.  [ ] Successfully ran forward pass in Transformers that gives\n   

### Creation of Retriever Tool

In [12]:
from smolagents import Tool

class RetrieverTool(Tool):
    name = "retriever"
    description = "Uses semantic search to retrieve the parts of transformers documentation that could be most relevant to answer your query."
    inputs = {
        "query": {
            "type": "string",
            "description": "The query to perform. This should be semantically close to your target documents. Use the affirmative form rather than a question.",
        }
    }
    output_type = "string"

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.retriever = db.as_retriever(search_kwargs={"k": 4})

    def forward(self, query: str) -> str:
        assert isinstance(query, str), "Your search query must be a string"

        docs = self.retriever.invoke(
            query,
        )
        return "\nRetrieved documents:\n" + "".join(
            [
                f"\n\n===== Document {str(i)} =====\n" + doc.page_content
                for i, doc in enumerate(docs)
            ]
        )

retriever_tool = RetrieverTool()

### Agent Initialization

In [13]:
from smolagents import HfApiModel, CodeAgent

agent = CodeAgent(
    tools=[retriever_tool], model=HfApiModel(), max_steps=4, verbose=True
)

In [14]:
agent_output = agent.run("For a transformers model training, which is slower, the forward or the backward pass?")

print("Final output:")
print(agent_output)

╭────────────────────────────────────────────────────────────────────────────────────────────── New run ───────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                                                                                                      │
│ For a transformers model training, which is slower, the forward or the backward pass?                                                                                                                │
│                                                                                                                                                                                                      │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 0 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Thought: To determine whether the forward or backward pass is slower during the training of a transformers model, I will use the `retriever` tool to find relevant information from the transformers    
documentation.                                                                                                                                                                                          
                                                                                                                                                                                                        
Code:                                                                                                                                                                                                   
```py                                                                                                                                                                                                   
forward_pass_info = retriever(query="forward pass in transformers")                                                                                                                                     
backward_pass_info = retriever(query="backward pass in transformers")                                                                                                                                   
print("Forward Pass Info:", forward_pass_info)                                                                                                                                                          
print("Backward Pass Info:", backward_pass_info)                                                                                                                                                        
```<end_code>                                                                                                                                                                                           

─ Executing this code: ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── 
  forward_pass_info = retriever(query="forward pass in transformers")                                                                                                                                   
  backward_pass_info = retriever(query="backward pass in transformers")                                                                                                                                 
  print("Forward Pass Info:", forward_pass_info)                                                                                                                                                        
  print("Backward Pass Info:", backward_pass_info)                                                                                                                                                      
 ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Forward Pass Info: 
Retrieved documents:


===== Document 0 =====
4.  [ ] Created script that successfully runs forward pass using
    original repository and checkpoint

5.  [ ] Successfully opened a PR and added the model skeleton to Transformers

6.  [ ] Successfully converted original checkpoint to Transformers
    checkpoint

7.  [ ] Successfully ran forward pass in Transformers that gives
    identical output to original checkpoint

8.  [ ] Finished model tests in Transformers

9.  [ ] Successfully added Tokenizer in Transformers

===== Document 1 =====
4.  [ ] Created script that successfully runs forward pass using
    original repository and checkpoint

5.  [ ] Successfully opened a PR and added the model skeleton to Transformers

6.  [ ] Successfully converted original checkpoint to Transformers
    checkpoint

7.  [ ] Successfully ran forward pass in Transformers that gives
    identical output to original checkpoint

8.  [ ] Finished model tests in Transformers

9.  [ ] Successfully added Tokenizer in Transformers

===== Document 2 =====
It is very likely that the 🤗 Transformers implementation and the original model implementation don't give the exact
same output the very first time or that the forward pass throws an error. Don't be disappointed - it's expected! First,
you should make sure that the forward pass doesn't throw any errors. It often happens that the wrong dimensions are
used leading to a *Dimensionality mismatch* error or that the wrong data type object is used, *e.g.* `torch.long`

===== Document 3 =====
It is very likely that the 🤗 Transformers implementation and the
original model implementation don't give the exact same output the very
first time or that the forward pass throws an error. Don't be
disappointed - it's expected! First, you should make sure that the
forward pass doesn't throw any errors. It often happens that the wrong
dimensions are used leading to a `"Dimensionality mismatch"`
error or that the wrong data type object is used, *e.g.*, `torch.long`
Backward Pass Info: 
Retrieved documents:


===== Document 0 =====
4.  [ ] Created script that successfully runs forward pass using
    original repository and checkpoint

5.  [ ] Successfully opened a PR and added the model skeleton to Transformers

6.  [ ] Successfully converted original checkpoint to Transformers
    checkpoint

7.  [ ] Successfully ran forward pass in Transformers that gives
    identical output to original checkpoint

8.  [ ] Finished model tests in Transformers

9.  [ ] Successfully added Tokenizer in Transformers

===== Document 1 =====
4.  [ ] Created script that successfully runs forward pass using
    original repository and checkpoint

5.  [ ] Successfully opened a PR and added the model skeleton to Transformers

6.  [ ] Successfully converted original checkpoint to Transformers
    checkpoint

7.  [ ] Successfully ran forward pass in Transformers that gives
    identical output to original checkpoint

8.  [ ] Finished model tests in Transformers

9.  [ ] Successfully added Tokenizer in Transformers

===== Document 2 =====
It is very likely that the 🤗 Transformers implementation and the original model implementation don't give the exact
same output the very first time or that the forward pass throws an error. Don't be disappointed - it's expected! First,
you should make sure that the forward pass doesn't throw any errors. It often happens that the wrong dimensions are
used leading to a *Dimensionality mismatch* error or that the wrong data type object is used, *e.g.* `torch.long`

===== Document 3 =====
It is very likely that the 🤗 Transformers implementation and the
original model implementation don't give the exact same output the very
first time or that the forward pass throws an error. Don't be
disappointed - it's expected! First, you should make sure that the
forward pass doesn't throw any errors. It often happens that the wrong
dimensions are used leading to a `"Dimensionality mismatch"`
erro

...

[Step 3: Duration 3.44 seconds| Input tokens: 14,289 | Output tokens: 401]

Reached max steps.

Final answer: ChatCompletionOutputMessage(role='assistant', content='In the context of training transformer models, the backward pass is generally slower than the forward pass. This is due to several 
reasons:\n\n1. **Complexity of Operations**: The backward pass involves computing gradients for each parameter in the model, which requires additional computations such as the chain rule in 
backpropagation. This results in more complex operations compared to the forward pass, which is primarily focused on computing the output.\n\n2. **Memory Usage**: The backward pass requires storing 
intermediate activations from the forward pass to compute gradients correctly. This additional memory usage can slow down the backward pass, especially for large models and long sequences.\n\n3. 
**Parallelization**: The forward pass can often be more easily parallelized across multiple GPUs or TPUs, whereas the backward pass can be more challenging to parallelize efficiently due to the 
dependencies between computations.\n\n4. **Gradient Accumulation**: In practice, the backward pass is often more time-consuming because it involves accumulating gradients over multiple forward passes 
before updating the model parameters. This accumulation step is necessary for training stable and effective models, especially with large batch sizes.\n\nWhile the exact time difference can vary 
depending on the specific model architecture, the computational load, and the hardware used, it is generally accepted that the backward pass is slower than the forward pass in transformer model 
training.', tool_calls=None)

[Step 4: Duration 0.00 seconds| Input tokens: 17,714 | Output tokens: 660]

Final output:
ChatCompletionOutputMessage(role='assistant', content='In the context of training transformer models, the backward pass is generally slower than the forward pass. This is due to several reasons:\n\n1. **Complexity of Operations**: The backward pass involves computing gradients for each parameter in the model, which requires additional computations such as the chain rule in backpropagation. This results in more complex operations compared to the forward pass, which is primarily focused on computing the output.\n\n2. **Memory Usage**: The backward pass requires storing intermediate activations from the forward pass to compute gradients correctly. This additional memory usage can slow down the backward pass, especially for large models and long sequences.\n\n3. **Parallelization**: The forward pass can often be more easily parallelized across multiple GPUs or TPUs, whereas the backward pass can be more challenging to parallelize efficiently due to the dependencies between co

In [15]:
print(agent_output.content)

In the context of training transformer models, the backward pass is generally slower than the forward pass. This is due to several reasons:

1. **Complexity of Operations**: The backward pass involves computing gradients for each parameter in the model, which requires additional computations such as the chain rule in backpropagation. This results in more complex operations compared to the forward pass, which is primarily focused on computing the output.

2. **Memory Usage**: The backward pass requires storing intermediate activations from the forward pass to compute gradients correctly. This additional memory usage can slow down the backward pass, especially for large models and long sequences.

3. **Parallelization**: The forward pass can often be more easily parallelized across multiple GPUs or TPUs, whereas the backward pass can be more challenging to parallelize efficiently due to the dependencies between computations.

4. **Gradient Accumulation**: In practice, the backward pass i

In [16]:
agent.run("For a transformers model training, What is the role of scaled dot product?")

╭────────────────────────────────────────────────────────────────────────────────────────────── New run ───────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                                                                                                      │
│ For a transformers model training, What is the role of scaled dot product?                                                                                                                           │
│                                                                                                                                                                                                      │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 0 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Thought: To understand the role of scaled dot product in a transformers model, I will use the `retriever` tool to fetch relevant information from the transformers documentation.                       
                                                                                                                                                                                                        
Code:                                                                                                                                                                                                   
```py                                                                                                                                                                                                   
scaled_dot_product_info = retriever(query="role of scaled dot product in transformers")                                                                                                                 
print(scaled_dot_product_info)                                                                                                                                                                          
```<end_code>                                                                                                                                                                                           

─ Executing this code: ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── 
  scaled_dot_product_info = retriever(query="role of scaled dot product in transformers")                                                                                                               
  print(scaled_dot_product_info)                                                                                                                                                                        
 ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:

Retrieved documents:


===== Document 0 =====
The main building block of any transformer is a fully connected `nn.Linear` followed by a nonlinear activation `GeLU`.
The dot dot-product part of it, following the Megatron's paper notation, can be written as `Y = GeLU(XA)`, where `X` is 
an input vector, `Y` is the output vector, and `A` is the weight matrix.

If we look at the computation in matrix form, you can see how the matrix multiplication can be split between multiple GPUs:

===== Document 1 =====
## BetterTransformer

<Tip warning={true}>

Some BetterTransformer features are being upstreamed to Transformers with default support for native `torch.nn.scaled_dot_product_attention`. BetterTransformer still has a wider coverage than the 
Transformers SDPA integration, but you can expect more and more architectures to natively support SDPA in Transformers.

</Tip>

<Tip>

===== Document 2 =====
## PyTorch scaled dot product attention

PyTorch's [`torch.nn.functional.scaled_dot_product_attention`](https://pytorch.org/docs/master/generated/torch.nn.functional.scaled_dot_product_attention.html) (SDPA) can also call FlashAttention and 
memory-efficient attention kernels under the hood. SDPA support is currently being added natively in Transformers and is used by default for `torch>=2.1.1` when an implementation is available.

===== Document 3 =====
BetterTransformer also converts all attention operations to use the more memory-efficient [scaled dot product 
attention](https://pytorch.org/docs/master/generated/torch.nn.functional.scaled_dot_product_attention).

<Tip>

BetterTransformer is not supported for all models. Check this [list](https://huggingface.co/docs/optimum/bettertransformer/overview#supported-models) to see if a model supports BetterTransformer.

</Tip>

Out: None

...

[Step 3: Duration 2.68 seconds| Input tokens: 11,496 | Output tokens: 304]

Reached max steps.

Final answer: ChatCompletionOutputMessage(role='assistant', content="The scaled dot product is a key component of the self-attention mechanism in transformer models, which plays a crucial role in how 
the model processes and understands the input data, particularly in natural language processing tasks.\n\nIn the context of transformers, the self-attention mechanism allows the model to weigh the 
importance of different words in a sentence relative to each other. This is done by computing a score for each pair of words, which indicates how relevant one word is to another. The scaled dot 
product is a method used to compute these scores efficiently.\n\nHere's a step-by-step breakdown of how the scaled dot product works in the self-attention mechanism:\n\n1. **Query, Key, and Value 
Vectors**: Each word in the input sequence is transformed into three vectors: a query vector, a key vector, and a value vector. These vectors are typically produced by applying linear transformations 
(using weight matrices) to the input embeddings.\n\n2. **Dot Product**: The dot product between the query vector of one word and the key vector of another word is computed. This dot product gives a 
score that indicates how well the query vector of the first word aligns with the key vector of the second word.\n\n3. **Scaling**: The dot product scores are scaled by dividing them by the square root
of the dimensionality of the key vector. This scaling helps to keep the dot product scores within a reasonable range, which in turn helps to stabilize the training process.\n\n4. **Softmax**: The 
scaled dot product scores are then passed through a softmax function to convert them into a probability distribution. These probabilities represent the attention weights, indicating the importance of 
each word in the context of the current word.\n\n5. **Weighted Sum**: The value vectors are multiplied by the attention weights, and the results are summed to produce the final output vector for the 
current word. This output vector is a weighted sum of the value vectors of all the words in the input sequence, where the weights are determined by the attention mechanism.\n\nThe scaled dot product 
attention mechanism allows the transformer model to focus on different parts of the input sequence when making predictions, which is crucial for understanding the context and relationships between 
words in a sentence. This mechanism is fundamental to the success of transformer models in a wide range of tasks, including language translation, text summarization, and question answering.", 
tool_calls=None)

[Step 4: Duration 0.00 seconds| Input tokens: 13,726 | Output tokens: 771]

ChatCompletionOutputMessage(role='assistant', content="The scaled dot product is a key component of the self-attention mechanism in transformer models, which plays a crucial role in how the model processes and understands the input data, particularly in natural language processing tasks.\n\nIn the context of transformers, the self-attention mechanism allows the model to weigh the importance of different words in a sentence relative to each other. This is done by computing a score for each pair of words, which indicates how relevant one word is to another. The scaled dot product is a method used to compute these scores efficiently.\n\nHere's a step-by-step breakdown of how the scaled dot product works in the self-attention mechanism:\n\n1. **Query, Key, and Value Vectors**: Each word in the input sequence is transformed into three vectors: a query vector, a key vector, and a value vector. These vectors are typically produced by applying linear transformations (using weight matrices) to 